<a href="https://colab.research.google.com/github/Aayushktyagi/Generative_models/blob/master/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

'''
Autoencoder implementation using tensorflow
'''

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


#load mnist Datasets
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./MNIST",one_hot = True)

#hyper parameters:
num_input = 784
n_layer_1 = 256
n_layer_2 = 128
n_layer_3 = 64
n_epochs = 1000
n_learning_rate = 0.001
batch_size = 50
display_step = 50

#input placeholder
X = tf.placeholder(tf.float32 , [None,num_input])

#weights
weights = {
    'encoder_h1':tf.Variable(tf.random_normal([num_input,n_layer_1])),
    'encoder_h2':tf.Variable(tf.random_normal([n_layer_1,n_layer_2])),
    'encoder_h3':tf.Variable(tf.random_normal([n_layer_2 , n_layer_3])),
    'decoder_h1':tf.Variable(tf.random_normal([n_layer_3,n_layer_2])),
    'decoder_h2':tf.Variable(tf.random_normal([n_layer_2,n_layer_1])),
    'decoder_h3':tf.Variable(tf.random_normal([n_layer_1,num_input]))
}
bias = {
    'encoder_b1':tf.Variable(tf.zeros([n_layer_1])),
    'encoder_b2':tf.Variable(tf.zeros([n_layer_2])),
    'encoder_b3':tf.Variable(tf.zeros([n_layer_3])),
    'decoder_b1':tf.Variable(tf.zeros([n_layer_2])),
    'decoder_b2':tf.Variable(tf.zeros([n_layer_1])),
    'decoder_b3':tf.Variable(tf.zeros([num_input]))
}
def encoder(x , weights, bias):
    layer_1 = tf.add(tf.matmul(x,weights['encoder_h1']),bias['encoder_b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1,weights['encoder_h2']),bias['encoder_b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2 , weights['encoder_h3']),bias['encoder_b3'])
    layer_3 = tf.nn.sigmoid(layer_3)

    return layer_3

def decoder(x , weights , bias):
    layer_1 = tf.add(tf.matmul(x , weights['decoder_h1']) , bias['decoder_b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1 , weights['decoder_h2']),bias['decoder_b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2,weights['decoder_h3']),bias['decoder_b3'])
    layer_3 = tf.nn.relu(layer_3)

    return layer_3

encoder_op = encoder(X , weights,bias)
decoder_op = decoder(encoder_op , weights,bias)

y_pred = decoder_op
y_true = X
#cost function
cost = tf.reduce_mean(tf.pow(tf.subtract(y_true,y_pred),2))
optimizer = tf.train.AdamOptimizer(n_learning_rate).minimize(cost)

#initialization
init = tf.global_variables_initializer()
#Save model
saver = tf.train.Saver()
#Staring training
with tf.Session() as sess:
    sess.run(init)
    for i  in range(n_epochs):
        epoch_loss = 0
        for _ in range(int(mnist.train.num_examples / batch_size)):
            batch_x , _ = mnist.train.next_batch(batch_size)
            #Run optimizatio and loss
            _ , l = sess.run([optimizer,cost],feed_dict={X:batch_x})
            epoch_loss +=l
        print("epoch :{} , loss:{}".format(i,epoch_loss))
        #Display batch_loss
        # if i % display_step == 0 or i ==0:
        #     print("Epoch :{} , loss :{}".format(i,l))
    save_path = saver.save(sess,"Model/Autoencoder.ckpt")
    correct = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true,1)) # y' = y then correct
    accuracy = tf.reduce_mean(tf.cast(correct, 'float')) #cast to float correct
    print('Accuracy: ',accuracy.eval({X:mnist.test.images, X:mnist.test.images})) #eval using x
	#test data
    index = np.random.randint(mnist.test.images.shape[0], size=8) #random 8 number from test data to be predicted
    autoencoder = sess.run(y_pred, feed_dict={X: mnist.test.images[index]}) #predicted test data using model
    for i in range(8):
    	plt.imsave("Model/Results"+str(i)+".png",np.reshape(autoencoder[i],(28,28)), cmap=plt.get_cmap('gray')) #save result


    #Starting testing
    # encode and decode images from test image set and visualization of reconstruction
    n = 4
    canvas_orignal = np.empty((28*n,28*n))
    canvas_recon = np.empty((28*n , 28*n))

    for i in range(n):
        #Get test dataset
        batch_x , _ = mnist.test.next_batch(n)
        g = sess.run(decoder_op , feed_dict = {X:batch_x})
        #Display  orignal
        for j in range(n):
            canvas_orignal[i*28:(i+1)*28,j*28:(j+1)*28] = batch_x[j].reshape([28,28])
            canvas_recon[i*28:(i+1)*28,j*28:(j+1)*28] = g[j].reshape([28,28])

    print("Orignal Images :")
    plt.figure(figsize = (n,n))
    plt.imshow(canvas_orignal,origin = "upper" , cmap = "gray")
    plt.show()
    plt.imsave("Model/Results_canvas_orignal"+".png",canvas_orignal)
    #plt.savefig("./Results/Results_canvas_orignal.png")

    print("Reconstructed Images :")
    plt.figure(figsize = (n,n))
    plt.imshow(canvas_recon,origin = "upper" , cmap = 'gray')
    plt.show()
    plt.imsave("Model/Results_canvas_recon"+".png",canvas_recon)
    # plt.savefig("./Results/Results_canvas_recon.png")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz
epoch :0 , loss:1568.5187359452248
epoch :1 , loss:118.0854884237051
epoch :2 , loss:115.83591171354055
epoch :3 , loss:115.1320595741272
epoch :4 , loss:114.40039288252592
epoch :5 , loss:113.66907443851233
epoch :6 , loss:112.96653579175472
epoch :7 , loss:112.10869196802378
epoch :8 , loss:111.35482232272625
epoch :9 , loss:110.83195753395557
epoch :10 , loss:110.3001536205411
epoch :11 , loss:109.8204782679677
epoch :12 , loss:109.45594964176416
epoch :13 , loss:109.12315694987774
epoch :14 , loss:108.82654456794262
epoch :15 , loss:108.5662847161293
epoch :16 , loss:108.34916514903307
epoch :17 , loss:108.17769184708595
epoch :18 , loss:108.04297047108412
epoch :19 , loss:1